In [1]:
import numpy as np
import pandas as pd
import glob
import re

try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds
import os
from tensorflow import keras

# PreProcessing

In [2]:
%ls

a structured self-attentive senctence embedding.pdf
pre_process.ipynb


In [3]:
with open("/Users/rainism/Desktop/grad/paper_to_code/data/NLP/prefix/nonbreaking_prefix.en",
          mode='r',
          encoding='utf-8') as f:
    non_breaking_prefix_en = f.read()

In [4]:
targetPattern = r"/Users/rainism/Desktop/grad/paper_to_code/data/NLP/pos/*.txt"
PATH = glob.glob(targetPattern)
ls = []

for i in PATH:
    with open(i,
              mode='r',
              encoding='utf-8') as f:
        ls.append(f.read())
        
        
targetPattern = r"/Users/rainism/Desktop/grad/paper_to_code/data/NLP/neg/*.txt"
PATH_neg = glob.glob(targetPattern)

for i in PATH_neg:
    with open(i,
              mode='r',
              encoding='utf-8') as f:
        ls.append(f.read())

In [5]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]

In [6]:
# Add $$$ after non ending sentence points
for prefix in non_breaking_prefix_en:
    for idx, corpus_en in enumerate(ls):
        corpus_en = corpus_en.replace(prefix, prefix + '$$$')
        ls[idx] = corpus_en
        
for idx, corpus_en in enumerate(ls):
    corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en) # ? means that weather it contains something such as [0-9],,,
    # # Remove $$$ markers
    corpus_en = re.sub(r".\$\$\$", '', corpus_en)
    # # Clear multiple spaces
    corpus_en = re.sub(r"  +", " ", corpus_en) # 스페이스 두개 이상(+) 일 경우, 공백 한개로 바꾼다.
    corpus_en = corpus_en.split('\n')
    ls[idx] = corpus_en

In [7]:
for idx, i in enumerate(ls):
    ls[idx] = "".join(i)

In [8]:
flat_list = ls

In [9]:
# tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
#     flat_list, target_vocab_size=2**13)

In [10]:
# VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2 # = 8190 # 여기에 2개를 넣는 이유는, <sos> , <eos> 이 들어갈거기 때문이다(?)
# inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]   # tokenizer_en.encode 하는게 sentence_to_token이네 // 하나 이해가 안가는것은, vocab_size_en -2 와 vocab_size_en-1을 왜하는지???
#           for sentence in ls]


## tokenizer setting

In [11]:
MAX_VOCAB_SIZE = 20000

tokenizer = tf.keras.preprocessing.text.Tokenizer(MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(flat_list)

In [12]:
word2vec = {}
EMBEDDING_DIM = 100
with open(os.path.join(r'/Users/rainism/Desktop/grad/paper_to_code/data/NLP/embedding/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
    for line in f:
        values = line.split()
        word2vec[values[0]] = values[1:]

In [13]:
word2vec['the'][:4]

['-0.038194', '-0.24487', '0.72812', '-0.39961']

## empty embedding matrix

In [14]:
word2idx = tokenizer.index_word

In [15]:
len(word2idx)

43296

In [16]:
NUM_WORDS = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrics = np.zeros((NUM_WORDS, EMBEDDING_DIM))

In [17]:
word2idx = tokenizer.word_index

for word, i in word2idx.items():
    if i < NUM_WORDS:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrics[i] = embedding_vector

In [18]:
sequences = tokenizer.texts_to_sequences(ls)

In [19]:
embedding_layer = tf.keras.layers.Embedding(
    NUM_WORDS,
    EMBEDDING_DIM,
    weights = [embedding_matrics],
    trainable=False
)

In [45]:
queries = tf.ragged.constant(sequences)

In [24]:
queries.shape

TensorShape([2000, None])

In [25]:
tmp = tf.random.normal([1,2,3])
tf.transpose(tmp)

<tf.Tensor: shape=(3, 2, 1), dtype=float32, numpy=
array([[[ 2.0299904 ],
        [-0.83025885]],

       [[ 0.93912446],
        [-0.63245714]],

       [[ 0.9670171 ],
        [-2.370026  ]]], dtype=float32)>

In [26]:
embedding_layer(queries[0]).shape

TensorShape([788, 100])

In [83]:
input_ = keras.Input(shape = (None,), ragged=True)
x = input_.to_tensor()
x = embedding_layer(x)
x1 = layers.LSTM(100, return_sequences=True, activation = 'tanh')(x)
x2 = layers.LSTM(100, return_sequences=True, activation = 'tanh', go_backwards = True)(x)
x = tf.concat([x1,x2], axis = -1) # n * 200
# x = tf.transpose(x)



Ws1 = tf.Variable(tf.random.normal([100,200]), shape=[100,200]) 
Ws2 = tf.Variable(tf.random.normal([5,100]), shape = [5,100]) # 5명의 사람
x_tanh = tf.nn.tanh(tf.matmul(Ws1, x, transpose_b=True))  # 100 * n
                    
                    
x_tanh = tf.matmul(Ws2, x_tanh) # 5 * n
x_softmax = tf.nn.softmax(x_tanh) # 5*n

                    
x = tf.matmul(x_softmax, x) # 5 * 200
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(10)(x)
x = keras.layers.Dense(2)(x)

model = keras.Model(input_, x)

model.summary()

The following Variables were used a Lambda layer's call (tf.linalg.matmul_2), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(100, 200) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.linalg.matmul_3), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(5, 100) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)        

In [ ]:

model.compile(
    
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)
answer = [1] * 1000 + [0] * 1000
answer = keras.utils.to_categorical(answer)
# answer = tf.ragged.constant(answer)
model.fit(queries, answer)

 1/63 [..............................] - ETA: 3:39 - loss: 4.2301 - accuracy: 0.5000

# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/ragged_tensor.ipynb#scrollTo=pHls7hQVJlk5

# BATCH

In [237]:
# BATCH_SIZE = 64
# BUFFER_SIZE = 20000

# dataset = tf.data.Dataset.from_tensor_slices((inputs, answer))
# dataset = dataset.cache()
# dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Let's build model

In [312]:
# class embedding_layer(layers.Layer):
    
#     def __init__(self,
#                  vocab_size,
#                  d_model):
#         super(embedding_layer, self).__init__()
        
#         self.vocab_size = vocab_size
#         self.d_model = d_model
        

    
#     def call(self, inputs):
        
#         embedding = layers.Embedding(self.vocab_size, self.d_model) 
#         print(embedding(tf.cast(inputs, dtype = float)).shape)
#         return embedding(tf.cast(inputs, dtype=float))

In [313]:
# class Bidirectional_LSTM(layers.Layer):
    
#     ''' bidirectional lstm을 정의한다. '''
    
#     def __init__(self, 
#                  units,
#                 vocab_size):
#         super(Bidirectional_LSTM, self).__init__()
        
        
#         self.units = units        # let's say it's 50
#         self.vocab_size = vocab_size
        
#     def build(self, input_shape):
        
#         self.d_model = input_shape[-1] # 100
#         self.sequence_length = input_shape[-2]
        
#         self.embedding = embedding_layer(self.vocab_size, 50)
        
#         # forward lstm
#         self.lstm_1 = layers.LSTM(self.units, return_sequences=True, activation = 'tanh')
#         # reverse lstm
#         self.lstm_2 = layers.LSTM(self.units, return_sequences=True, activation = 'tanh')
    
        
#     def call(self, inputs, outputs):

#         x = self.embedding(inputs)
#         print(x.shape)
#         x1 = self.lstm_1(x)
#         x2 = self.lstm_1(x)
#         x = tf.concat([x1,x2], axis = -1)

#         return x

In [314]:
bi = Bidirectional_LSTM(50, VOCAB_SIZE_EN)

In [315]:
bi(inputs[0],answer[0])

(1111, 50)
(1111, 50)


ValueError: Input 0 of layer lstm_8 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (1111, 50)